In [13]:

import requests
import os
from dotenv import load_dotenv
import pandas as pd
import time

load_dotenv()

EIA_KEY = os.getenv("EIA_KEY")

In [14]:
url = "https://api.eia.gov/v2/electricity/rto/region-data/data"
all_data = []

for offset in range(0, 20000, 5000):  # Fetch 4 pages: 0, 5000, 10000, 15000
    params = {
        "api_key": EIA_KEY,
        "frequency": "hourly",
        "data[0]": "value",
        "facets[type][]": "D",
        "facets[respondent][]": "ERCO",
        "start": "2024-01-01T00",
        "end": "2024-12-31T00",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": offset,
        "length": 5000
    }

    print(f"Fetching records {offset} to {offset+5000}...")
    response = requests.get(url, params=params)
    if response.status_code == 200:
        batch = response.json().get("response", {}).get("data", [])
        all_data.extend(batch)
        if len(batch) < 5000:
            break  # No more data
    else:
        print("Failed at offset", offset)
        print(response.text)
        break

    time.sleep(1)  # Pause to be polite to the API

print(f"Total records collected: {len(all_data)}")

Fetching records 0 to 5000...
Fetching records 5000 to 10000...
Total records collected: 8761


In [19]:
df_EIA_Demand = pd.DataFrame(all_data)

In [ ]:
df_EIA_Demand

,period,respondent,respondent-name,type,type-name,value,value-units
0,2024-12-31T00,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,50098,megawatthours
1,2024-12-30T23,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,50334,megawatthours
2,2024-12-30T22,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,50245,megawatthours
3,2024-12-30T21,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,49602,megawatthours
4,2024-12-30T20,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,48595,megawatthours
...,...,...,...,...,...,...,...
8756,2024-01-01T04,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,43279,megawatthours
8757,2024-01-01T03,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,44012,megawatthours
8758,2024-01-01T02,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,44887,megawatthours
8759,2024-01-01T01,ERCO,"Electric Reliability Council of Texas, Inc.",D,Demand,45855,megawatthours


In [21]:
df_EIA_Demand.to_csv("./data/raw/EIA_Demand.csv")

In [37]:
EIA_clean_df = df_EIA_Demand.drop(axis=0, columns={'respondent-name', 'type', 'value-units', 'respondent', 'type-name'})

In [38]:
EIA_clean_df.rename(columns={'value':'demand-mwh'}, inplace=True)
EIA_clean_df

,period,demand-mwh
0,2024-12-31T00,50098
1,2024-12-30T23,50334
2,2024-12-30T22,50245
3,2024-12-30T21,49602
4,2024-12-30T20,48595
...,...,...
8756,2024-01-01T04,43279
8757,2024-01-01T03,44012
8758,2024-01-01T02,44887
8759,2024-01-01T01,45855


In [39]:
EIA_clean_df['period'] = pd.to_datetime(EIA_clean_df['period'],utc=True )
EIA_clean_df = EIA_clean_df.set_index('period').sort_index()
EIA_clean_df

,demand-mwh
period,
2024-01-01 00:00:00+00:00,44394
2024-01-01 01:00:00+00:00,45855
2024-01-01 02:00:00+00:00,44887
2024-01-01 03:00:00+00:00,44012
2024-01-01 04:00:00+00:00,43279
...,...
2024-12-30 20:00:00+00:00,48595
2024-12-30 21:00:00+00:00,49602
2024-12-30 22:00:00+00:00,50245


In [50]:
EIA_clean_df['demand-kwh'] = pd.to_numeric(EIA_clean_df['demand-kwh'], errors='coerce')

KeyError: 'demand-kwh'

In [45]:
daily_demand = EIA_clean_df.resample('D').sum()
print(daily_demand.head())

                                                                  demand-mwh
period                                                                      
2024-01-01 00:00:00+00:00  4439445855448874401243279423184168141371416314...
2024-01-02 00:00:00+00:00  4688849045492554945249015481304676746073455184...
2024-01-03 00:00:00+00:00  5368354163538345245951056489994677845545445344...
2024-01-04 00:00:00+00:00  4931951141511445083349917481924658645626451304...
2024-01-05 00:00:00+00:00  4889650486504004969948482465734463843279425074...


In [51]:
EIA_clean_df.to_csv('./data/raw/test.csv')